# Hello World: handwritten digit recognition with SuperDuperDB

In this tutorial, we implement the classic MNIST handwritten digits recognition task in SuperDuperDB. 

First we import the SuperDuperDB client, and create a fresh collection.

In [1]:
from superduperdb.client import the_client
import torchvision

docs = the_client.mnist.digits

The data is available as `PIL.Image` images with labels in the `torchvision` package:

In [2]:
import random

mnist_data = list(torchvision.datasets.MNIST(root='./data', download=True))
random.shuffle(mnist_data)

SuperDuperDB is based on MongoDB, which does not support images and tensors and other special data types out of the box. In order to remedy this, we create custom SuperDuperDB **types**.
These types handle:
    
- How to store images as bytes in SuperDuperDB and how to reinstantiate the images from the bytes
- How to store tensors in SuperDuperDB and how to retrieve these from the database again

We do this by creating the classes in python with `.encode` and `.decode` methods:

In [3]:
!pygmentize examples/types.py

import io
import numpy
import PIL.Image
import PIL.JpegImagePlugin
import torch


class Image:
    types = (PIL.JpegImagePlugin.JpegImageFile, PIL.Image.Image)

    @staticmethod
    def encode(x):
        buffer = io.BytesIO()
        x.save(buffer, format='png')
        return buffer.getvalue()

    @staticmethod
    def decode(bytes_):
        return PIL.Image.open(io.BytesIO(bytes_))


class FloatTensor:
    types = (torch.FloatTensor, torch.Tensor)

    @staticmethod
    def encode(x):
        x = x.numpy()
        assert x.dtype == numpy.float32
        return memoryview(x).tobytes()

    @staticmethod
    def decode(bytes_):
        array = numpy.frombuffer(bytes_, dtype=numpy.float32)
        return torch.from_numpy(array).type(torch.float)


Once these classes are ready, one can import the classes, and add instances of these to SuperDuperDB, giving each type a suitable name:

In [4]:
from examples.types import Image, FloatTensor

docs.create_type('image', Image)
docs.create_type('float_tensor', FloatTensor)

Now we have these custom types, we can insert the data into SuperDuperDB:

In [5]:
docs.insert_many([{'img': x[0], 'class': x[1]} for x in mnist_data[:-1000]])

downloading content from retrieved urls
found 0 urls


You'll see that when we fetch data from the database, it's in exactly the form that we want.

In [6]:
docs.find_one()

{'_id': ObjectId('640939c0deb71bb414652022'),
 'img': <PIL.PngImagePlugin.PngImageFile image mode=L size=28x28>,
 'class': 9,
 '_fold': 'train'}

In [7]:
docs.find_one()['img']

Now that we've added the data to SuperDuperDB, we're ready to create a model. This is a simple PyTorch model implementing the iconic [LeNet architecture](https://en.wikipedia.org/wiki/LeNet). In addition to the standard PyTorch `.forward` method, SuperDuperDB allows users to specify `.preprocess` and `.postprocess` methods which define respectively:

- how the data is converted from the in-database form, to tensor
- from the output form back into the form to be stored in the database

In [8]:
!pygmentize examples/mnist/models.py

import torch
from torchvision import transforms


def label(x):
    return torch.tensor(x)


class LeNet5(torch.nn.Module):
    def __init__(self, num_classes):
        super().__init__()
        self.layer1 = torch.nn.Sequential(
            torch.nn.Conv2d(1, 6, kernel_size=5, stride=1, padding=0),
            torch.nn.BatchNorm2d(6),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2))
        self.layer2 = torch.nn.Sequential(
            torch.nn.Conv2d(6, 16, kernel_size=5, stride=1, padding=0),
            torch.nn.BatchNorm2d(16),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2))
        self.fc = torch.nn.Linear(400, 120)
        self.relu = torch.nn.ReLU()
        self.fc1 = torch.nn.Linear(120, 84)
        self.relu1 = torch.nn.ReLU()
        self.fc2 = torch.nn.Linear(84, num_classes)

    def preprocess(self, x):
        return transforms.Compose([
            transforms.Resize((32, 32)),
            tran

Let's test this model on a single image. `Collection.apply_model` applies serially the `preprocess`, `forward` and `postprocess` methods, creating a singleton batch prior to the `forward` and unpacking the batch after the `forward`.

In [9]:
from examples.mnist.models import LeNet5, label

docs.apply_model(LeNet5(10), docs.find()[23]['img'])

2

Now that we have our model we need a target for learning - for this we use `label`:

In [10]:
docs.create_model('lenet', LeNet5(10))
docs.create_function('label', label)

Finally we need a loss function - this is a vanilla `torch` function, so we don't need to write extra code:

In [11]:
import torch

docs.create_objective('classification', torch.nn.CrossEntropyLoss())

And a metric to measure performance:

In [2]:
!pygmentize examples/mnist/metrics.py

def accuracy(x, y):
    return x == y


In [12]:
from examples.mnist.metrics import accuracy

docs.create_metric('accuracy', accuracy)

When measuring performance, SuperDuperDB requires users to create a separate validation set, which is saved for posterity and reproducibility reasons - edits and deletes on the main collection don't affect this validation set.

In [13]:
docs.create_validation_set('classification', sample_size=250)

  0%|          | 0/250 [00:00<?, ?it/s]

downloading content from retrieved urls
found 0 urls


Now we're ready to create the model using the `Collection.create_imputation`. This trains a model to predict one part of the data using another part, specified respectively by `model_key` and `target_key`; these are subkeys of the collection documents.

In [14]:
docs.create_imputation(
    name='predictor',
    model='lenet',
    model_key='img',
    target='label',
    target_key='class',
    objective='classification',
    metrics=['accuracy'],
    n_iterations=1000,
    validation_interval=50,
    validation_sets=('classification',)
)

downloading ids for {'_fold': 'train'}


  0%|          | 0/56016 [00:00<?, ?it/s]

downloading records for {'_fold': 'valid'}


  0%|          | 0/2984 [00:00<?, ?it/s]

  0%|          | 0/250 [00:00<?, ?it/s]

...

  0%|          | 0/5000 [00:00<?, ?it/s]

bulk writing...
done.
computing chunk (12/12)
finding documents under filter
done.
processing with lenet


  0%|          | 0/4000 [00:00<?, ?it/s]

bulk writing...
done.


[None]

After training, you'll see that a model **watcher** has been created, which keeps the `img` key up-to-date

In [15]:
docs.list_watchers()

[('lenet', 'img')]

When new data are added, the trained model kicks into action 
and it's outputs are added/ enriched to the newly added data

In [18]:
docs.insert_many([{'img': x[0], 'class': x[1]} for x in mnist_data[-1000:]])

downloading content from retrieved urls
found 0 urls
computing chunk (1/1)
finding documents under filter
done.
processing with lenet


  0%|          | 0/1000 [00:00<?, ?it/s]

bulk writing...
done.
